# Process cell testing data
*Version / date: 01.08.2018*

*By: Jan Petter Mæhlen*

## Short description of the notebook
This notebook is intended for people / students that only need to look at a limited set of cell cycling tests and don´t need to use the full power of ``cellpy`` ([look at the Read the Docs page for more info](https://cellpy.readthedocs.io/en/latest/)). The basic steps consists of:
1. Save the raw-data files for the set of tests that you would like to examine in a folder (`raw_data_folder`).
2. Fill in the necessery data in the information dictionary (`info_dict`)
    1. where you have your raw data
    2. where you would like to put the processed data and figures
    3. information about the tests, such as what group they belong to (e.g. the duplicate cells could belong to the same group), and the masses of the electrodes.
3. Perform the loading and the extraction of the cycling data.
4. You can then treat the data further utilising for example the [Pandas](https://pandas.pydata.org/) library or make interactive plots using [Bokeh](https://bokeh.pydata.org/en/latest/).

**Remark!** The current version of the notebook utilises functions and modules that will very soon become deprecated (when the new batch-method is ready for production).

## Preparing

### Importing the needed Python modules

In [ ]:
import os
import logging

import pandas as pd

from cellpy import prms
from cellpy import log
from cellpy.utils import batch
import cellpy.utils.plotutils as plot_utils

In [ ]:
%load_ext autoreload
%autoreload 2
log.setup_logging(default_level="INFO")

### Information about files and folders

In [ ]:
# Provide the name of the folder where your raw-data are located (all the files needs to be in the same folder)
# Hint! If you are working on an windows computer where the back-slash is used for
# dividing folders, you can use raw string format (i.e. r"C:\some\folder").

raw_data_folder = "/Users/jepe/scripting/cellpy/testdata/data"

# Provider the name of the folder where you would like to store your processed data and figures.

output_folder = "/Users/jepe/scripting/tmp"

In [ ]:
assert os.path.isdir(raw_data_folder)
assert os.path.isdir(output_folder)
print("Congratulations, both folders exists", end=" ")
prms.Paths.outdatadir = output_folder
prms.Paths.rawdatadir = raw_data_folder
print("- giving this info to cellpy.")

In [ ]:
# Provide a list-of-lists of your raw-files (where tests that consists of more than one file are 
# bundled inside another list)

raw_files = [
    ["20160805_test001_45_cc_01.res", "20160805_test001_45_cc_02.res"],
    ["20160805_test001_46_cc_01.res"],
    ["20160805_test001_47_cc_01.res"],
]

In [ ]:
# Also provide a name for the set of tests / batch (e.g. "experiment_001")
# and what project it belongs to (e.g. "H2020_binders")

batch_name = "try01"       # The name will be used for naming sub-folders
project_name = "tmp_proj"  # The project name will be used for making (or selecting) the main out-put folder

In [ ]:
json_file, (project_folder, batch_folder, output_raw_folder) = batch.create_folder_structure(project_name, batch_name)

In [ ]:
print(f"project_folder:        {project_folder}\n"
      f"batch_folder:          {batch_folder}\n"
      f"output_raw_folder:     {output_raw_folder}\n"
      f"suggested json name:   {json_file}\n"
)

### Information about the cells

In [ ]:
masses = [
    0.12,
    0.33,
    0.27,
]

groups = [
    1,
    1,
    2,
]


cycle_mode = "anode"

### Running loading and saving routines

In [ ]:
prms.Reader.cycle_mode = cycle_mode

raw_file_paths = list()
_cell_names = list()
_empty = list()
_true = list()
_cell_types = list()

for duplicates in raw_files:
    raw_file_paths.append([os.path.join(raw_data_folder, f) for f in duplicates])
    _cell_names.append(duplicates[0].split(".")[0])
    _empty.append(None)
    _true.append(True)
    _cell_types.append(cycle_mode)

cell_names = _cell_names
    
info_dict = dict()
info_dict["filenames"] = cell_names
info_dict["masses"] = masses
info_dict["total_masses"] = _empty
info_dict["loadings"] = _empty
info_dict["fixed"] = _true
info_dict["labels"] = _empty
info_dict["cell_type"] = _cell_types
info_dict["raw_file_names"] = raw_file_paths
info_dict["cellpy_file_names"] = _empty
info_dict["groups"] = groups

info_df = pd.DataFrame(info_dict)
info_df = info_df.sort_values(["groups", "filenames"])
info_df = batch._make_unique_groups(info_df)
info_df["labels"] = info_df["filenames"].apply(batch.create_labels)
info_df.set_index("filenames", inplace=True)

In [ ]:
info_df.head()

In [ ]:
frames, keys, errors = batch.read_and_save_data(info_df, output_raw_folder,
                                               force_raw=True,
                                               force_cellpy=False,
                                               export_cycles=True,
                                               export_ica=True,
                                               use_cellpy_stat_file=False,
                                               save=False)

In [ ]:
if len(errors):
    print(errors)

In [ ]:
# b.make_summaries()
selected_summaries = [
    "discharge_capacity", "charge_capacity",
    "coulombic_efficiency",
    "cumulated_coulombic_efficiency",
    "ir_discharge", "ir_charge",
    "end_voltage_discharge",
    "end_voltage_charge",
]

summary = batch.save_summaries(frames, keys, selected_summaries, batch_folder, batch_name)

In [ ]:
symbol_label = prms.Batch.symbol_label
color_style_label = prms.Batch.color_style_label
figure_type = prms.Batch.figure_type

color_list, symbol_list = plot_utils.create_colormarkerlist_for_info_df(info_df, symbol_label=symbol_label, 
                                                                        color_style_label=color_style_label)

fig, ax = batch.plot_summary_figure(info_df, summary, color_list, symbol_list, selected_summaries, batch_folder,
                                    batch_name, show=True, save=False, figure_type=figure_type)

## Going further with your data

In [ ]:
summary45 = summary['20160805_test001_45_cc_01']
summary45.plot(y=["IR_Charge(Ohms)", "IR_Discharge(Ohms)"])